## Installing packages and Imports

In [1]:
!pip install ollama
!python -m pip install langchain==0.1.0 langchain-community==0.0.12 langchainhub==0.1.14
!pip install faiss-gpu
!pip install faiss-cpu
# !pip install langchain-core

  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached anyio-4.3.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
Using cached anyio-4.3.0-py3-none-any.whl (85 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached PyYAML-6.0.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached SQLAlchemy-2.0.30-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached aiohttp-3.9.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.5 kB)
  Using cached dataclasses_json-0.6.6-py3-none-any.whl.metadata (25 kB)
  Using cached jsonpatch-1.33-py2.py3-

In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama

from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

## Ollama Client

### Option 1: Run Ollama server locally
**In this approach, you need resources to run the Ollama server locally. Better to run it on Google Colab.**

In [6]:
# Install colab-xterm package and load the extension for running a terminal in Colab
!pip install colab-xterm
%load_ext colabxterm

The colabxterm extension is already loaded. To reload it, use:
  %reload_ext colabxterm


**Run the following cell and run these two commands in the Xterm:**

```sh
curl -fsSL https://ollama.com/install.sh | sh
ollama serve & ollama pull llama2:7b
```

In [ ]:
%xterm

In [ ]:
OLLAMA_BASE_URL = "http://localhost:11434" 
OLLAMA_MODEL = "llama2"

### Option 2: Connect to a remote server
This option gives you access to a remote Ollama server, thus you can run this code on your low-resources local machine.  
**However, it would be slower due to the requests time to a remote server.**

In [26]:
OLLAMA_BASE_URL = "https://63ab-34-125-22-255.ngrok-free.app"
OLLAMA_MODEL = "llama3"

## Load dataset

In [14]:
REVIEWS_PATH="data/reviews.csv"

In [15]:
loader = CSVLoader(file_path=REVIEWS_PATH, source_column="text.text",
                    encoding = 'utf-8', metadata_columns= ["place_id", "rating"])
documents = loader.load()

### Get some insight and clean the dataset

In [27]:
# @title Get some insight on a sample document
def get_insight(docs):
  anomalies = []
  print(f"docs.page_content:\n{docs[0].page_content}")
  print(f"\ndocs.metadata:\n{docs[0].metadata}")
  m = 0
  for doc in docs:
    review = doc.metadata["source"]
    l = len(review)
    if l > m:
        m = l
    if l == 3427:
      print(review)
    if l >= 2040:
      anomalies.append(doc.metadata["row"])
  print(f"\nMaximum length of reviews in the dataset: {m}")
  if m < 2048:
    print(f"Maximum length of review < 2048, so we're good!")
  else:
    print("Maximum length of review >= 2028, so we're not good!")
  return anomalies

anomalies = get_insight(documents)

docs.page_content:
name: places/ChIJpXSgrsDbfkcRzf_5kCMmrZI/reviews/ChdDSUhNMG9nS0VJQ0FnSUNUdnYyMnBBRRAB
text.languageCode: en
text.text: Huge "eating palace" which is lacking the fine accents of the italian food although the service was top. I would recommend for lunch, less for dinner

docs.metadata:
{'source': 'Huge "eating palace" which is lacking the fine accents of the italian food although the service was top. I would recommend for lunch, less for dinner', 'row': 0, 'place_id': 'ChIJpXSgrsDbfkcRzf_5kCMmrZI', 'rating': '4'}

Maximum length of reviews in the dataset: 1700
Maximum length of review < 2048, so we're good!


In [28]:
documents.remove(documents[anomalies[-1]])

IndexError: list index out of range

In [18]:
anomalies = get_insight(documents)

docs.page_content:
name: places/ChIJpXSgrsDbfkcRzf_5kCMmrZI/reviews/ChdDSUhNMG9nS0VJQ0FnSUNUdnYyMnBBRRAB
text.languageCode: en
text.text: Huge "eating palace" which is lacking the fine accents of the italian food although the service was top. I would recommend for lunch, less for dinner

docs.metadata:
{'source': 'Huge "eating palace" which is lacking the fine accents of the italian food although the service was top. I would recommend for lunch, less for dinner', 'row': 0, 'place_id': 'ChIJpXSgrsDbfkcRzf_5kCMmrZI', 'rating': '4'}

Maximum length of reviews in the dataset: 1700
Maximum length of review < 2048, so we're good!


Take a look at this [link](https://api.python.langchain.com/en/latest/embeddings/langchain_community.embeddings.ollama.OllamaEmbeddings.html#langchain-community-embeddings-ollama-ollamaembeddings) and this [link](https://api.python.langchain.com/en/latest/llms/langchain_community.llms.ollama.Ollama.html) for details of models parameters.

In [29]:
store = LocalFileStore("./cache/")
# can increase num_ctx up to 4,096 tokens!
embedding_model = OllamaEmbeddings(model=OLLAMA_MODEL, num_ctx=2048, temperature=0, base_url=OLLAMA_BASE_URL)

In [30]:
embedder = CacheBackedEmbeddings.from_bytes_store(
    embedding_model, store, namespace=embedding_model.model)

In [31]:
# smaple embedding
text = "This is a test document."

query_result = embedder.embed_query(text)
query_result[:5], len(query_result)

([-0.5911625623703003,
  -5.051942825317383,
  0.16411244869232178,
  -0.3182397782802582,
  -0.049503616988658905],
 4096)

API reference [FAISS](https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.faiss.FAISS.html#langchain_community.vectorstores.faiss.FAISS.similarity_search)

In [32]:
vector_db = FAISS.from_documents(documents, embedder,
                                 distance_strategy="EUCLIDEAN_DISTANCE") # "COSINE"

KeyboardInterrupt: 

In [ ]:
docs = vector_db.similarity_search("which one is the best pizza restaurant in the city?",
                                      k = 5)

In [ ]:
docs[0]

Document(page_content='name: places/ChIJ4aOff1rafkcRLpeSS2KAsPI/reviews/ChZDSUhNMG9nS0VJQ0FnSURqeU92Q0lBEAE\ntext.languageCode: en\ntext.text: Pizza was very and home made. Staff was fast, efficient and organized. I recommend this place!', metadata={'source': 'Pizza was very and home made. Staff was fast, efficient and organized. I recommend this place!', 'row': 607, 'place_id': 'ChIJ4aOff1rafkcRLpeSS2KAsPI', 'rating': '4'})

In [ ]:
docs = vector_db.similarity_search_with_score("which one is the best pizza restaurant in the city?",
                                      k = 5, )

In [ ]:
docs[0]

(Document(page_content='name: places/ChIJ4aOff1rafkcRLpeSS2KAsPI/reviews/ChZDSUhNMG9nS0VJQ0FnSURqeU92Q0lBEAE\ntext.languageCode: en\ntext.text: Pizza was very and home made. Staff was fast, efficient and organized. I recommend this place!', metadata={'source': 'Pizza was very and home made. Staff was fast, efficient and organized. I recommend this place!', 'row': 607, 'place_id': 'ChIJ4aOff1rafkcRLpeSS2KAsPI', 'rating': '4'}),
 8951.218)

In [ ]:
docs[4]

(Document(page_content="name: places/ChIJu9bZt0_afkcRsYoU1BCI1SI/reviews/ChZDSUhNMG9nS0VJQ0FnSUNUdUltVEl3EAE\ntext.languageCode: en\ntext.text: After a third visit to Padova and several fAfter three visits to Padova and several unsuccessful attempts, we finally managed to find a table in this wonderful osteria. The place has a charming, family-run feel with a lot of heart. It's cosy, quite and comfortable.\r\nThe food was brilliant, the wine was great, and the grappa was so tasty!\r\nWhen you travel to Padova, make sure to visit this place!\r\n(hard to find a free table, so make a reservation).", metadata={'source': "After a third visit to Padova and several fAfter three visits to Padova and several unsuccessful attempts, we finally managed to find a table in this wonderful osteria. The place has a charming, family-run feel with a lot of heart. It's cosy, quite and comfortable.\r\nThe food was brilliant, the wine was great, and the grappa was so tasty!\r\nWhen you travel to Padova, mak

In [ ]:
REVIEWS_FAISS_PATH = "faiss_index"
FAISS_INDEX_NAME = "index"
vector_db.save_local(folder_path=REVIEWS_FAISS_PATH, index_name=FAISS_INDEX_NAME)
vector_db = FAISS.load_local(folder_path=REVIEWS_FAISS_PATH, embeddings=embedder, index_name=FAISS_INDEX_NAME)

`db.as_retriever` has some cool options!

In [ ]:
# Retrieve more documents with higher diversity
# Useful if your dataset has many similar documents
vector_db.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 6, 'lambda_mult': 0.25}
)

# Fetch more documents for the MMR algorithm to consider
# But only return the top 5
vector_db.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 5, 'fetch_k': 50}
)

# Only retrieve documents that have a relevance score
# Above a certain threshold
vector_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.8}
)

# Only get the single most similar document from the dataset
vector_db.as_retriever(search_kwargs={'k': 1})

# Use a filter to only retrieve documents from a specific paper
vector_db.as_retriever(
    search_kwargs={'filter': {'paper_title':'GPT-4 Technical Report'}}
)

## Creating chatbot

In [ ]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

from langchain_core.output_parsers import StrOutputParser
# from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough

review_template_str = """Your job is to use Google Map
reviews to answer questions about their experience at a restaurant. Use
the following context to answer questions. Be as detailed as possible, but
don't make up any information that's not from the context. If you don't know
an answer based on the context, say you don't know.
context:
{context}
"""
## """
# If you don't know an answer based on the context, say you don't know, and
# if the context is not about restaurants, then kindly tell them that  you can
# only provide assistance and answer questions related to restaurants.
##"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)
messages = [review_system_prompt, review_human_prompt]

review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"], messages=messages
)

chat_model = Ollama(model="llama2", temperature=0.)

output_parser = StrOutputParser()

# REVIEWS_CHROMA_PATH = "chroma_data/"
# reviews_vector_db = Chroma(
#     persist_directory=REVIEWS_CHROMA_PATH,
#     embedding_function=OpenAIEmbeddings(),
# )
reviews_vector_db = vector_db = FAISS.load_local(folder_path=REVIEWS_FAISS_PATH,
                                         embeddings=embedder,
                                         index_name=FAISS_INDEX_NAME)

reviews_retriever = reviews_vector_db.as_retriever(k=10)

review_chain = (
    {"context": reviews_retriever, "question": RunnablePassthrough()}
    | review_prompt_template
    | chat_model
    | StrOutputParser()
)

In [ ]:
question = """What are the pros and cons of the best pizza restaurant in the city?"""
review_chain.invoke(question)

'Based on the Google Map reviews provided, here are some pros and cons of the best pizza restaurant in the city:\n\nPros:\n\n1. Delicious food: The restaurant serves delicious pizzas that are well-liked by customers.\n2. Good portions: The restaurant offers good portions of food, including their pizzas.\n3. Organized staff: The staff is well-organized and efficient, ensuring a smooth dining experience.\n4. Recommendation: Many reviewers have recommended this restaurant to others.\n\nCons:\n\n1. Crowded during lunchtime: The restaurant can get crowded during lunchtime, which may be inconvenient for some customers.\n2. Limited seating: The restaurant has limited seating capacity, which can lead to long wait times for a table.\n3. Difficulty finding a free table: It can be challenging to find a free table at the restaurant, especially during peak hours.\n4. No reservation system: The restaurant does not have a reservation system in place, which can make it difficult for customers to secur

In [ ]:
s = """Based on the Google Map reviews provided, here are some pros and cons of the best pizza restaurant in the city:

Pros:

1. Delicious food: The restaurant serves delicious pizzas that are well-liked by customers.
2. Good portions: The restaurant offers good portions of food, including their pizzas.
3. Organized staff: The staff is well-organized and efficient, ensuring a smooth dining experience.
4. Recommendation: Many reviewers have recommended this restaurant to others.

Cons:

1. Crowded during lunchtime: The restaurant can get crowded during lunchtime, which may be inconvenient for some customers.
2. Limited seating: The restaurant has limited seating capacity, which can lead to long wait times for a table.
3. Difficulty finding a free table: It can be challenging to find a free table at the restaurant, especially during peak hours.
4. No reservation system: The restaurant does not have a reservation system in place, which can make it difficult for customers to secure a table without waiting.

Overall, the best pizza restaurant in the city seems to have both positive and negative aspects. While the food is delicious and the staff is well-organized, the crowded atmosphere during lunchtime and limited seating capacity may be drawbacks for some customers."""

In [ ]:
# from langchain.agents import create_openai_functions_agent, Tool, AgentExecutor
# from langchain import hub
# from langchain_intro.tools import get_current_wait_time

tools = [
    Tool(
        name="Reviews",
        func=review_chain.invoke,
        description="""Useful when you need to answer questions
        about patient reviews or experiences at the hospital.
        Not useful for answering questions about specific visit
        details such as payer, billing, treatment, diagnosis,
        chief complaint, hospital, or physician information.
        Pass the entire question as input to the tool. For instance,
        if the question is "What do patients think about the triage system?",
        the input should be "What do patients think about the triage system?"
        """,
    ),
    Tool(
        name="Waits",
        func=get_current_wait_time,
        description="""Use when asked about current wait times
        at a specific hospital. This tool can only get the current
        wait time at a hospital and does not have any information about
        aggregate or historical wait times. This tool returns wait times in
        minutes. Do not pass the word "hospital" as input,
        only the hospital name itself. For instance, if the question is
        "What is the wait time at hospital A?", the input should be "A".
        """,
    ),
]

hospital_agent_prompt = hub.pull("hwchase17/openai-functions-agent")

agent_chat_model = ChatOpenAI(
    model="gpt-3.5-turbo-1106",
    temperature=0,
)

hospital_agent = create_openai_functions_agent(
    llm=agent_chat_model,
    prompt=hospital_agent_prompt,
    tools=tools,
)

hospital_agent_executor = AgentExecutor(
    agent=hospital_agent,
    tools=tools,
    return_intermediate_steps=True,
    verbose=True,
)